# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,new norfolk,-42.7826,147.0587,30.70,30,61,1.80,AU,1736472686
1,1,crane,31.3974,-102.3501,4.66,57,11,4.99,US,1736472687
2,2,port-aux-francais,-49.3500,70.2167,4.34,92,100,23.35,TF,1736472688
3,3,utrik,11.2278,169.8474,26.79,78,6,9.55,MH,1736472690
4,4,arraial do cabo,-22.9661,-42.0278,23.94,80,96,4.73,BR,1736472691


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display 

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",               # Longitude on the x-axis
    "Lat",               # Latitude on the y-axis
    size="Humidity",     # Size of the points represents humidity
    color="City",        # Color the points uniquely for each city
    alpha=0.6,           # Add transparency for better visibility
    frame_width=700,     # Set the map width
    frame_height=500,    # Set the map height
    geo=True,            # Enable geographic mapping
    tiles="OSM",         # Use OpenStreetMap tiles for the map
    legend="right"       # Display the legend with city names on the right
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
109,109,saint-pierre,-21.3393,55.4781,24.82,73,0,2.57,RE,1736472815
148,148,port elizabeth,-33.9180,25.5701,21.16,89,0,2.57,ZA,1736472859
254,254,manaus,-3.1019,-60.0250,25.28,88,0,1.03,BR,1736472990
285,285,piton saint-leu,-21.2196,55.3151,23.06,73,0,2.57,RE,1736473028
429,429,beltran,-27.8291,-64.0610,22.22,68,0,1.36,AR,1736473199


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
109,saint-pierre,RE,-21.3393,55.4781,73,
148,port elizabeth,ZA,-33.9180,25.5701,89,
254,manaus,BR,-3.1019,-60.0250,88,
285,piton saint-leu,RE,-21.2196,55.3151,73,
429,beltran,AR,-27.8291,-64.0610,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,  # Limit the results to 1 hotel per request
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
saint-pierre - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
manaus - nearest hotel: No hotel found
piton saint-leu - nearest hotel: No hotel found
beltran - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
109,saint-pierre,RE,-21.3393,55.4781,73,No hotel found
148,port elizabeth,ZA,-33.9180,25.5701,89,No hotel found
254,manaus,BR,-3.1019,-60.0250,88,No hotel found
285,piton saint-leu,RE,-21.2196,55.3151,73,No hotel found
429,beltran,AR,-27.8291,-64.0610,68,No hotel found
520,bubaque,GW,11.2833,-15.8333,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
ideal_map = hotel_df.hvplot.points(
    "Lng",               # Longitude on the x-axis
    "Lat",               # Latitude on the y-axis
    size="Humidity",     # Size of the points represents humidity
    color="City",        # Color the points uniquely for each city
    alpha=0.6,           # Add transparency for better visibility
    frame_width=700,     # Set the map width
    frame_height=500,    # Set the map height
    geo=True,            # Enable geographic mapping
    tiles="OSM",         # Use OpenStreetMap tiles for the map
    legend="right",      # Display the legend with city names on the right
    hover_cols=["Hotel Name", "Country"]  # Add Hotel Name and Country to the hover information
)

# Display the map
ideal_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)